In [ ]:
#problem16: Finding a Protein Motif
#패턴 찾기 함수, 웹으로부터 파싱, 웹에 요청 실패 응답코드
#별다섯개 다시 볼것

import requests
import time

#sample id 꺼내오기
with open("rosalind_mprt.txt", "r") as sample_file:
    uniprot_ids = [line.strip() for line in sample_file]

#print(uniprot_id)

#N{P}{ST}{P} 모티프 검색 함수
def find_motif(sequence):
    positions = []
    for i in range(len(sequence)-3):
        string = sequence[i:i+4]
        if (
            string[0] == "N" and
            string[1] != "P" and
            string[2] in 'ST' and
            string[3] != "P"
        ):
            positions.append(i+1) #1 based index
    return positions

#id마다 처리
for uniprot_id in uniprot_ids:
    url_id = uniprot_id.split('_')[0]
    url = f"http://www.uniprot.org/uniprot/{url_id}.fasta"
    
    time.sleep(0.3) #딜레이 안 넣으면 서버 부담 때문에 전부 데이터가 안나옴
    response = requests.get(url)
    
    #오류 때문에 파싱이 안되는 경우가 있음
    if response.status_code != 200:
        print(f"{uniprot_id}: Failed to fetch (status code {response.status_code})")
        continue

    lines = response.text.strip().split('\n')
    sequence = ''.join(lines[1:]) #header 제외

    positions = find_motif(sequence)
    if positions:
        print(uniprot_id)
        print(' '.join(map(str, positions)))



P81428_FA10_TROCA
254
P14210_HGF_HUMAN
294 402 566 653
P13838_LEUK_RAT
274 300
P28314_PER_COPCI
162
P17404_CHM1_BOVIN
244
P10761_ZP3_MOUSE
146 273 304 327 330
P80069_A45K_MYCBO
7 161
Q00001_RHGA_ASPAC
50 235 317
P47002
35 552 608


In [ ]:
#problem17: Inferring mRNA from Protein
# 모듈러로 100만 처리, 즉 백만 이상은 버림

protein_cnt = {
    "F": 2, "L": 6, "S": 6, "Y": 2, "C": 2, "W": 1,
    "P": 4, "H": 2, "Q": 2, "R": 6, "I": 3, "*": 3,
    "M": 1, "T": 4, "N": 2, "K": 2, "V": 4, "A" : 4,
    "D": 2, "E": 2, "G": 4
}

protein_seq = input("protein sequence")
def inferring_protein(protein_seq):
    result = 1
    for amino in protein_seq:
        if amino in protein_cnt:
            result *= protein_cnt[amino]
            result %= 1_000_000 #백만으로 나눈 나머지
    result*=protein_cnt["*"]
    result %= 1_000_000 #혹시 모르니
    return result

print(inferring_protein(protein_seq))

760384


In [ ]:
#problem18: Open Reading Frames
#여러 함수를 만들고 쓰느라 왔다갔다 헷갈림
#다시 풀어볼 것

#단백질 코돈 딕셔너리
codon_table = {
    "UUU": "F", "UUC": "F",
    "UUA": "L", "UUG": "L", "CUU": "L", "CUC": "L", "CUA": "L", "CUG": "L",
    "AUU": "I", "AUC": "I", "AUA": "I",
    "AUG": "M",
    "GUU": "V", "GUC": "V", "GUA": "V", "GUG": "V",
    "UCU": "S", "UCC": "S", "UCA": "S", "UCG": "S", "AGU": "S", "AGC": "S",
    "CCU": "P", "CCC": "P", "CCA": "P", "CCG": "P",
    "ACU": "T", "ACC": "T", "ACA": "T", "ACG": "T",
    "GCU": "A", "GCC": "A", "GCA": "A", "GCG": "A",
    "UAU": "Y", "UAC": "Y",
    "CAU": "H", "CAC": "H",
    "CAA": "Q", "CAG": "Q",
    "AAU": "N", "AAC": "N",
    "AAA": "K", "AAG": "K",
    "GAU": "D", "GAC": "D",
    "GAA": "E", "GAG": "E",
    "UGU": "C", "UGC": "C",
    "UGG": "W",
    "CGU": "R", "CGC": "R", "CGA": "R", "CGG": "R", "AGA": "R", "AGG": "R",
    "GGU": "G", "GGC": "G", "GGA": "G", "GGG": "G",
    "UAA": "*", "UAG": "*", "UGA": "*"
}


#DNA 서열 읽어들이기
DNA_seq = {}

with open("rosalind_orf.txt") as handle:
    for line in handle:
        line = line.strip()
        if line.startswith(">"):
            dna_id = line[1:]
            DNA_seq[dna_id] = ""
        else:
            DNA_seq[dna_id] += line

#DNA -> RNA transcription
RNA_seq = {}
for k, v in DNA_seq.items():
    RNA_seq[k] = v.replace("T", "U")
#RNA_seq = {k: v.replace("T", "U") for k, v in DNA_seq.items()}

#역상보RNA 만들기
def reverse_complement(dna):
    reverse_base  = {"A": "T", "T": "A", "G": "C", "C": "G"}
    return ''.join(reverse_base[base] for base in reversed(dna))

# ORF translation 함수

def translate_single_rna(rna_seq):
    global codon_table
    amino_acid_seq = []
    read_length = len(rna_seq)
    for i in range(read_length-2):
        codon = rna_seq[i:i+3]
        if codon == "AUG": #M부터 시작
            protein_seq = ""
            have_stop = False
            for j in range(i, read_length, 3):
                codon_j = rna_seq[j:j+3]
                if len(codon_j) < 3:
                    break
                aa = codon_table.get(codon_j)
                if aa == "*" or aa is None:
                    have_stop = True
                    amino_acid_seq.append(protein_seq)
                    break
                protein_seq += aa
            if have_stop:
                amino_acid_seq.append(protein_seq)
    return amino_acid_seq        


# 6 reading freame

def find_6frames(dna_seq_dict):
    all_proteins = []
    for seq in dna_seq_dict.values():
        #정방향
        for frame in range(3):
            rna = seq[frame:].replace("T", "U")
            all_proteins.extend(translate_single_rna(rna))
        #역방향
        rev_comp = reverse_complement(seq)
        for frame in range(3):
            rna = rev_comp[frame:].replace("T", "U")
            all_proteins.extend(translate_single_rna(rna))

    return all_proteins
    

proteins = find_6frames(DNA_seq)
for protein in set(proteins):
    print(protein)

MVLE
MYSV
MEDQLAVHRRTCGRGSSTGPCTHLVWSNLQTENWLFEGPWLTIN
MRSESPTRLRELALVSLVGEEITRASQLLRPAVYPCIGVTP
MDRPPHVVLAFL
MCS
MNTL
MGRHGVHQFIVNQGPSNNQFSVWRLDQTRCVQGPVEEPRPQVRR
MTATLTPYVTILVIYEYTLALTFVHSSALPPEPFQVT
MCNSELQGRLTNL
MVQTALEANRLR
MVYISLSSTKVLQTTSSQFGGWTRLDACKDPWKNRVHRFDGERPVDLPYVIPI
MQGYTAGLRSCEALVISSPTRLTRASSLSLVGDSLRIIYFGPRCIYL
MHRGPK
MPPHT
MYTMPPHT
MRA
M
MRARTRGRTASTGSTVNGQLIFHM
MMVQTALEANRLR


In [ ]:
#problem19: Enumerating Gene Orders
#백트랙이나 재귀함수로 풀어보기
#itertools 사용하면 너무 쉬운듯

n = int(input("n <= 7"))
while n > 7:
    n = int(input("n <= 7"))

#순열 함수
def permutation(n):
    result = 1
    for _ in range(n, 0, -1):
        result *= n
        n -= 1
    return result

base = [x for x in range(1, n+1)]
#print(base)

#itertools 쓴 버전
from itertools import permutations
all_perms = list(permutations(base))
print(len(all_perms))

for perm in all_perms:
    print(' '.join(map(str, perm)))

    

5040
1 2 3 4 5 6 7
1 2 3 4 5 7 6
1 2 3 4 6 5 7
1 2 3 4 6 7 5
1 2 3 4 7 5 6
1 2 3 4 7 6 5
1 2 3 5 4 6 7
1 2 3 5 4 7 6
1 2 3 5 6 4 7
1 2 3 5 6 7 4
1 2 3 5 7 4 6
1 2 3 5 7 6 4
1 2 3 6 4 5 7
1 2 3 6 4 7 5
1 2 3 6 5 4 7
1 2 3 6 5 7 4
1 2 3 6 7 4 5
1 2 3 6 7 5 4
1 2 3 7 4 5 6
1 2 3 7 4 6 5
1 2 3 7 5 4 6
1 2 3 7 5 6 4
1 2 3 7 6 4 5
1 2 3 7 6 5 4
1 2 4 3 5 6 7
1 2 4 3 5 7 6
1 2 4 3 6 5 7
1 2 4 3 6 7 5
1 2 4 3 7 5 6
1 2 4 3 7 6 5
1 2 4 5 3 6 7
1 2 4 5 3 7 6
1 2 4 5 6 3 7
1 2 4 5 6 7 3
1 2 4 5 7 3 6
1 2 4 5 7 6 3
1 2 4 6 3 5 7
1 2 4 6 3 7 5
1 2 4 6 5 3 7
1 2 4 6 5 7 3
1 2 4 6 7 3 5
1 2 4 6 7 5 3
1 2 4 7 3 5 6
1 2 4 7 3 6 5
1 2 4 7 5 3 6
1 2 4 7 5 6 3
1 2 4 7 6 3 5
1 2 4 7 6 5 3
1 2 5 3 4 6 7
1 2 5 3 4 7 6
1 2 5 3 6 4 7
1 2 5 3 6 7 4
1 2 5 3 7 4 6
1 2 5 3 7 6 4
1 2 5 4 3 6 7
1 2 5 4 3 7 6
1 2 5 4 6 3 7
1 2 5 4 6 7 3
1 2 5 4 7 3 6
1 2 5 4 7 6 3
1 2 5 6 3 4 7
1 2 5 6 3 7 4
1 2 5 6 4 3 7
1 2 5 6 4 7 3
1 2 5 6 7 3 4
1 2 5 6 7 4 3
1 2 5 7 3 4 6
1 2 5 7 3 6 4
1 2 5 7 4 3 6
1 2 5 7 4 6 3
1 2 5 7 6 3 4
1

In [6]:
#problem20: Calculating Protein Mass

monoisotopic_mass_table = {
    "A" : 71.03711, "C" : 103.00919, "D" : 115.02694, "E" : 129.04259,
    "F" : 147.06841, "G" : 57.02146, "H" : 137.05891, "I" : 113.08406,
    "K" : 128.09496, "L" : 113.08406, "M" : 131.04049, "N" : 114.04293,
    "P" : 97.05276, "Q" : 128.05858, "R" : 156.10111, "S" : 87.03203,
    "T" : 101.04768, "V" : 99.06841, "W" : 186.07931, "Y" : 163.06333
}

polypeptide = input("sequence")

weight = 0
for aa in polypeptide:
    if aa in monoisotopic_mass_table:
        weight += monoisotopic_mass_table[aa]
    
print(round(weight, 3))

102784.182
